In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
#convertir texto en un valor a una matrix, cada columna es una palabra
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import unicodedata
import pickle
import re
from sklearn.metrics import classification_report
from sklearn import metrics

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [3]:
df.sample(10)

,url,is_spam
2377,https://www.morningbrew.com/daily/r,True
1128,https://www.vulture.com/2020/06/zadie-smith-ne...,False
1821,https://theneed2know.us2.list-manage.com/about,True
2858,https://numlock.substack.com/p/numlock-sunday-...,True
1743,https://woven.com/,False
1350,https://www.reuters.com/article/us-unilever-wh...,False
2278,https://www.theskimm.com/,True
2295,https://www.theskimm.com/picks/13-of-the-hotte...,True
1315,https://expmag.com/2020/06/health-tracking-imp...,False
739,https://www.hvper.com/,True


In [4]:
#Eliminamos los caracteres desconocidos
df['url']=df['url'].str.strip()
df['url']=df['url'].str.lower()
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [5]:
df['url']=df['url'].str.replace('!','')
df['url']=df['url'].str.replace(',','')
df['url']=df['url'].str.normalize('NFKC') #ANALIZA TEXTO NO LATINO Y LO TRATA DE ARREGLAR
df['url']=df['url'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)
df.sample(10)

,url,is_spam
2282,https://w.npr.org/2019/10/04/763863712/supreme...,False
2611,https://w.bloomberg.com/tosv2.html,True
2512,https://w.nytimes.com/article/maskne-acne.html,False
2178,https://digg.com/editions/top_stories_6_30_20,False
99,https://docs.google.com/forms/d/e/1faipqlscc99...,True
996,https://w.outsideonline.com/2415069/sperm-whal...,False
1146,https://tim.blog/2016/04/05/caroline-paul/,False
129,https://w.forbes.com/sites/jenniferwang/2020/0...,False
1972,https://w.bookdepository.com/love-colour-bolu-...,False
1742,https://w.qmunity.tech/qubes-camp,False


In [6]:
def normalize_string(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else: 
        result=None
    return result

df['url']=df['url'].apply(normalize_string)
df.sample(10)

,url,is_spam
1149,https://tim.blog/,True
1701,https://w.foodandwine.com/travel/restaurants/s...,False
604,https://shop.morningbrew.com/,True
1366,https://w.morningbrew.com/daily/refer-a-friend,True
1532,https://bigspaceship.us9.list-manage.com/unsub...,True
1608,https://w.forbes.com/sites/martyswant/2020/06/...,False
980,https://news.trust.org/item/20200624154945-q8zfe/,False
2328,https://w.cnbc.com/2020/06/28/tock-market-futu...,False
863,https://drinkolipop.com/,False
430,https://w.reuters.com/article/us-saudi-haj/sau...,False


In [139]:
X=df['review']
y=df['polarity']